In [1]:
!nvidia-smi

Thu May 19 06:50:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install -U synonyms

     |████████████████████████████████| 10.7 MB 5.2 MB/s 
  Created wheel for synonyms: filename=synonyms-3.16.0-py3-none-any.whl size=10832785 sha256=ec2d073d36c798722ca1938348365e3576e5964403ef59106b0e5418cbcd43bf
  Stored in directory: /root/.cache/pip/wheels/e4/cd/43/b4548753509a94471fc946967a07116252d49aeeb689db8f7c
Successfully built synonyms


In [4]:
! pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.


In [5]:
! pip install transformers==4.0.1

     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 880 kB 38.2 MB/s 
     |████████████████████████████████| 2.9 MB 34.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d2183a0fbdf8fcc9b869e4d22337a472e0e87d7badeedf73c5369086e752d667
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [6]:
import synonyms

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/usr/local/lib/python3.7/dist-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/usr/local/lib/python3.7/dist-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz] ...

[Synonyms] downloading data from https://github.com/chatopera/Synonyms/releases/download/3.15.0/words.vector.gz to /usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz ... 
 this only happens if SYNONYMS_WORD2VEC_BIN_URL_ZH_CN is not present and Synonyms initialization for the first time. 
 It would take minutes that depends on network.

[Synonyms] downloaded.



## EDA（Easy Data Augmentation）
![EDA3](https://img-blog.csdnimg.cn/50c22b4212714b509ce053ff921d6bdd.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 对于训练集中的给定句子，随机选择并执行以下操作之一：
* 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
* 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
* 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
* 随机删除（RD）：以概率 p 随机删除句子中的每个单词。

In [7]:
# 读取停用词表
import random
import re
from random import shuffle
# strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
stop_words = {word.strip() for word in open('/content/drive/MyDrive/Colab Notebooks/dataset/baidu_stopwords.txt', 'r', encoding='utf8').readlines()}

In [8]:
# 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。

# 得到同义词列表
def get_synonym(word):
  # nearby返回一个元组，位置0返回同义词，位置1返回相似度
  sys = set(synonyms.nearby(word)[0])
  # 去除原词
  if word in sys:
    sys.remove(word)
  return list(sys)
  # 如果输入"给力" 可能没有同义词（同义词只有他自己） 则返回  ([],[])

# 同义词替换
# 传入一个词列表和替换词的数量
def synonym_replacement(words, n):

  new_words = words.copy()
  # 去除停用词，去重，变成列表
  random_word_list = list(set([word for word in words if word not in stop_words]))
  # 打乱
  random.shuffle(random_word_list)

  num_replaced = 0
  for random_word in random_word_list:
    synonym_words = get_synonym(random_word)
    if len(synonym_words)>=1:
        # 随机选取一个同义词
        synonym = random.choice(list(synonym_words))
        # 如果word是要替换的词 替换成同义词 ，否则返回原词
        new_words = [synonym if word == random_word else word for word in new_words]
        # 同义词替换数量+1
        num_replaced += 1
        
    if num_replaced >= n:
        break
  # 为什么加这两句话？ 看note1
  sentence = ' '.join(new_words)
  new_words = sentence.split(' ')

  return new_words

In [9]:
# 使用eda进行数据增强
# 同义词替换的比例、 增强的句子数目
def eda(sentence, alpha_sr=0.1, num_aug=9):

  # 位置0是词组、1是词性
  words = synonyms.seg(sentence)[0]
  num_words = len(words)
  n_sr = max(1, int(alpha_sr * num_words))

  augmented_sentences = []

  for _ in range(num_aug):
    a_words = synonym_replacement(words, n_sr)
    augmented_sentences.append(' '.join(a_words))
  return augmented_sentences

In [10]:
# note1
# 存在这样一种情况
sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'actor']
word = 'actor'
example_synonyms = ['actress', 'film star', 'performer', 'comedian', 'entertainer']
new_sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'film star']
# 为了消除空格
new_sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'film', 'star']

In [11]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

['9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 古巴 的 相应 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 古巴 控制技术 ， 而 无法 OEM 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 弊端 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭到 顽强抵抗 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 服务商 ， 只要 没有 新西兰 的 相关机构 证书 ， 即便 无法 供应 模组 给 华为 ， 而 中芯国际 等 国产 模组 企业 ， 也 因 采用 新西兰 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 沉重打击 。',
 '9 月 15 日 以来 ， 宏碁 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 前述 许可证 ， 即便 无法 供应 器件 给 华为 ， 而 中芯国际 等 国产 器件 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 商品 出现 货 少 的 弊端 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 顽强抵抗 。',
 '9 月 15 日 以来 ， Tektronix 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 墨西哥 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 SE9 等 改良版 芯片 企业 ， 也 因 采用 墨西哥 技术 ， 而 无法 接单 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 若 该 严峻形势 持续 下去 ， 华为 手机 业务 将 遭受重创 重创 。',
 '9 月 15 年 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 市场供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 跨国公司 ， 也 因 采用 美国 技术开发 ， 而 无法 发货 给 华为 。 目前 华为 部分 型号 的 手

![UDA5](https://img-blog.csdnimg.cn/9d10da70d1d0467e93ef5bb1267ac87f.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
**$ \tilde{\theta} $： 参数不变 不进行反向传播**


![在这里插入图片描述](https://img-blog.csdnimg.cn/88a3abe95bbd4e369fe4d085533c9c35.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [12]:
from bucket_sampler import SortedSampler, BucketBatchSampler
from EMA import *

In [13]:
import random
import numpy as np
import torch
config = {
        'train_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json',
        'dev_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/dev.json',
        'test_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/test.json',
        'output_path': '.',
        'model_path': '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
        'batch_size': 16,
        'num_epochs': 1,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'unsup_data_ratio': 1.5,
        'uda_softmax_temp': 0.4,
        'uda_confidence_threshold': 0.8,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 300,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2022
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

In [14]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [15]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids =True, return_attention_mask = True)

  inputs['input_ids'].append(inputs_dict['input_ids'])
  inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
  inputs['attention_mask'].append(inputs_dict['attention_mask'])
  inputs['labels'].append(label)

## 对偶数据增强

**a-b对，变成b-a对, 把两个句子换顺序**\
**我们的无监督数据增强就是用的对偶数据增强**\
**BERT输入 a，b两个句子，现在输入以b,a作为输入，增强样本**

In [16]:
import pandas as pd
from tqdm import tqdm
import json
def parse_data(path, data_type='train'):
  sentence_a = []
  sentence_b = []
  labels = []

  with open(path, 'r', encoding = 'utf8') as f:
    for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
      line = json.loads(line)
      sentence_a.append(line['sentence1'])
      sentence_b.append(line['sentence2'])
      if data_type != 'test':
        labels.append(int(line['label']))
      else:
        labels.append(0)

  df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns = ['text_a', 'text_b', 'labels'])
  return df

In [17]:
# 无监督BERT输入
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  # 左右
  lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)
  # 右左
  rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)

  # 元组的形式
  inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
  inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'], rl_inputs_dict['token_type_ids']))
  inputs['attention_mask'].append((lr_inputs_dict['attention_mask'], rl_inputs_dict['attention_mask']))
  inputs['labels'].append(label)

## startswith()方法
startswith() 方法用于检查字符串是否是以指定子字符串开头\
如果是则返回 True，否则返回 False。如果参数 beg 和 end 指定值，则在指定范围内检查。\
str.startswith(str, beg=0,end=len(string));\
*参数*
```
str --检测的字符串。
strbeg --可选参数用于设置字符串检测的起始位置。
strend --可选参数用于设置字符串检测的结束位置。
```

In [18]:
from collections import defaultdict
def read_data(config, tokenizer):
  train_df = parse_data(config['train_file_path'], data_type = 'train')
  dev_df = parse_data(config['dev_file_path'], data_type = 'dev')
  test_df = parse_data(config['test_file_path'], data_type = 'test')

  data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
  processed_data = {}
  unsup_data = defaultdict(list)
  for data_type, df in data_df.items():
    inputs = defaultdict(list)
    if data_type == 'train':
      reversed_inputs = defaultdict(list)

    for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total = len(df)):
      label = 0 if data_type == 'test' else row[2]
      sentence_a, sentence_b = row[0], row[1]
      build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)

      if data_type.startswith('test'):
        build_bert_inputs(inputs, label, sentence_b, sentence_a, tokenizer)


      build_unsup_bert_inputs(unsup_data, label, sentence_a, sentence_b, tokenizer)

    processed_data[data_type] = inputs
  processed_data['unsup_data'] = unsup_data
  return processed_data

# processed_data
# {
#    'train':,
#    'dev':,
#    'test':,
#    'unsup_data':   # 数据量最大的
# }

In [19]:
data = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:10<00:00, 385.31it/s]


In [20]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(AFQMCDataset, self).__init__()
    self.data_dict = data_dict

  def __getitem__(self, idx):
    data = (self.data_dict['input_ids'][idx],
         self.data_dict['token_type_ids'][idx],
         self.data_dict['attention_mask'][idx],
         self.data_dict['labels'][idx])
    return data
  
  def __len__(self):
    return len(self.data_dict['input_ids'])


In [21]:
class Collator:
  def __init__(self, max_seq_len, tokenizer):
    self.max_seq_len = max_seq_len
    self.tokenizer = tokenizer

  def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
    input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype = torch.long)
    token_type_ids = torch.zeros_like(input_ids)
    attention_mask = torch.zeros_like(input_ids)
    for i in range(len(input_ids_list)):
      seq_len = len(input_ids_list[i])
      if seq_len <= max_seq_len:
        input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype = torch.long)
        token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype = torch.long)
        attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype = torch.long)

      else:
        input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len - 1] + [self.tokenizer.sep_token_id], dtype = torch.long)
        token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype = torch.long)
        attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype = torch.long)

    labels = torch.tensor(labels_list, dtype = torch.long)
    return input_ids, token_type_ids, attention_mask, labels

  def __call__(self, examples):
    input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)

    input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)

    data_dict = {
        'input_ids':input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }
    return data_dict

In [22]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

In [23]:
# UDA 无监督重新构造dataset 和 collator
class UnsupAFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(UnsupAFQMCDataset, self).__init__()
    self.data_dict = data_dict
  
  def __getitem__(self, idx):
    input_ids = self.data_dict['input_ids'][idx]
    token_type_ids = self.data_dict['token_type_ids'][idx]
    attention_mask = self.data_dict['attention_mask'][idx]
    labels = self.data_dict['labels'][idx]
    # input_ids[0]：lr_inputs_dict['input_ids']    (build_unsup_bert_inputs)
    # input_ids[1]：rl_inputs_dict['input_ids']
    return (input_ids[0], token_type_ids[0], attention_mask[0],
         input_ids[1], token_type_ids[1], attention_mask[1],
         labels)
    
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [24]:
class UnsupCollator(Collator):
  def __init__(self, max_seq_len, tokenizer):
    super(UnsupCollator, self).__init__(max_seq_len, tokenizer)

  def __call__(self, examples):
    # 根据UnsupAFQMCDataset的getitem 有七个数据
    (ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list,
     ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list,
     labels_list) = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in ab_input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)
    # 分批整ab, ba（填充与截断）
    ab_input_ids, ab_token_type_ids, ab_attention_mask, labels = self.pad_and_truncate(ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list, labels_list, max_seq_len)
    ba_input_ids, ba_token_type_ids, ba_attention_mask, labels = self.pad_and_truncate(ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list, labels_list, max_seq_len)


    data_dict = {
        'ab_input_ids':ab_input_ids,
        'ab_token_type_ids':ab_token_type_ids,
        'ab_attention_mask':ab_attention_mask,
        'ba_input_ids':ba_input_ids,
        'ba_token_type_ids':ba_token_type_ids,
        'ba_attention_mask':ba_attention_mask,
        'labels':labels
    }
    return data_dict

In [25]:
from torch.utils.data import DataLoader, RandomSampler
def build_dataloader(config, data, tokenizer):
  train_dataset = AFQMCDataset(data['train'])
  dev_dataset = AFQMCDataset(data['dev'])
  test_dataset = AFQMCDataset(data['test'])
  unsup_dataset = UnsupAFQMCDataset(data['unsup_data'])

  collate_fn = Collator(config['max_seq_len'], tokenizer)
  unsup_collate_fn = UnsupCollator(config['max_seq_len'], tokenizer)

  # 使用桶采样
  if config['use_bucket']:
    # 监督数据
    # 基采样器RandomSampler
    train_sampler = RandomSampler(train_dataset)
    # drop_last 最后一个batch小于size 丢弃
    bucket_sampler = BucketBatchSampler(train_sampler, batch_size = config['batch_size'],
                      drop_last = False, sort_key = lambda x: len(train_dataset[x][0]),# 以 input_id 长度作为排序的指标
                      bucket_size_multiplier = config['bucket_multiplier'])
    train_dataloader = DataLoader(dataset = train_dataset, batch_sampler = bucket_sampler, num_workers = 4, collate_fn = collate_fn)
    # 无监督数据 
    # grad_data中(图) 有监督、无监督一起训练， 无监督数据量大， batchsize可以设置大一点
    unsup_sampler = RandomSampler(unsup_dataset)
    unsup_bucket_sampler = BucketBatchSampler(unsup_sampler, batch_size = int(config['batch_size'] * config['unsup_data_ratio']),
                      drop_last = False, sort_key = lambda x: len(unsup_dataset[x][0]),# 以 input_id 长度作为排序的指标
                      bucket_size_multiplier = config['bucket_multiplier'])
    
    unsup_dataloader = DataLoader(dataset = unsup_dataset, batch_sampler = unsup_bucket_sampler, num_workers = 4, collate_fn = unsup_collate_fn)
  # 不使用桶采样
  else:
    # 监督数据
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = config['batch_size'], shuffle = True, num_workers = 4, collate_fn = collate_fn)
    # 无监督数据
    unsup_dataloader = DataLoader(dataset = unsup_dataset, batch_size = int(config['batch_size'] * config['unsup_data_ratio']), 
                    shuffle = True, num_workers = 4, collate_fn = unsup_collate_fn)
  # 验证集、测试集dataloader 与桶采样无关 因为shuffle=false  
  dev_dataloader = DataLoader(dataset = dev_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)
  test_dataloader = DataLoader(dataset = test_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)

  return unsup_dataloader, train_dataloader, dev_dataloader, test_dataloader

In [26]:
unsup_dataloader, train_dataloader, dev_dataloader, test_dataloader = build_dataloader(config, data, tokenizer)

In [27]:
from sklearn.metrics import f1_score, accuracy_score
def evaluation(config, model, val_dataloader):
  model.eval()
  preds = []
  labels = []
  val_loss = 0.
  val_iterator = tqdm(val_dataloader, desc = 'Evaluation', total = len(val_dataloader))

  with torch.no_grad():
    for batch in val_iterator:
      labels.append(batch['labels'])
      batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
      batch_cuda['mode'] = 'val'
      loss, logits = model(**batch_cuda)[:2]

      if config['n_gpus'] > 1:
        loss = loss.mean()

      val_loss += loss.item()
      preds.append(logits.argmax(dim = -1).detach().cpu())

  avg_val_loss = val_loss / len(val_dataloader)
  labels = torch.cat(labels, dim = 0).numpy()
  preds = torch.cat(preds, dim = 0).numpy()
  f1 = f1_score(labels, preds)
  acc = accuracy_score(labels, preds)
  return avg_val_loss, f1, acc

In [28]:
from transformers import BertForSequenceClassification
import torch.nn as nn
class BertForAFQMC(BertForSequenceClassification):
   # 复写forward
   def forward(self, input_ids, token_type_ids, attention_mask, labels = None, mode= 'train'):

     outputs = self.bert(input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, output_hidden_states = True)
     # 维度：[batch_size, hidden_size]
     pooled_output = outputs[1]
     pooled_output = self.dropout(pooled_output)

     logits = self.classifier(pooled_output)
     # print('BertForAFQMC中logits：',logits)
     outputs = (logits, )
     # print('BertForAFQMC中outputs：',outputs)

     if mode == 'val':
       loss_fct = nn.CrossEntropyLoss()
       # X.view(-1)中的-1本意是根据另外一个数来自动调整维度
       loss = loss_fct(logits, labels.view(-1))

       outputs = (loss, ) + outputs
     return outputs

![addtitional Training1](https://img-blog.csdnimg.cn/5916fe8ae028469bb877d15a1ac566de.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
### 额外训练技巧
### Confidence-based masking用于无监督训练数据中
基于置信度的MASK，发现MASK当前模型不自信的examples很有帮助。总结来说，无监督数据（grad_data ba_unsup_value）要 选出置信度>$\beta$的样本（够自信的样本）

![UDA5](https://img-blog.csdnimg.cn/1ddf28077b88449aa84e0391149467e4.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

###  A.1 
在半监督学习中，经常会遇到未标记数据量和标记数据量存在巨大差异的情况\
模型通常很快会在标记数据上过拟合，同时在未标记数据欠拟合。\
为了解决这个问题，引入一种技术，训练信号退火（TSA）. 它随着训练的进行逐渐释放。

这是一种MASK\
当$\eta=1$ 代表所有数据都训练了\
当$\eta=0.5$  代表不是所有数据都训练

In [29]:
# 上方Figure 5公式
def get_tsa_threshold(total_steps, global_steps):
  return np.exp((global_steps / total_steps - 1) * 5) / 2 + 0.5

![在这里插入图片描述](https://img-blog.csdnimg.cn/88a3abe95bbd4e369fe4d085533c9c35.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
## 有监督数据和无监督数据怎么拼接：
### 无监督的batchsize大，但句子长度不一定大 图为else的情况
### 连接时先有监督后无监督，图有误（重要！）
**get_data**

![在这里插入图片描述](https://img-blog.csdnimg.cn/38a2b12d76094f17819ae918b25f3c71.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [30]:
# 返回 grad_data：需要计算梯度，需要进行反向传播的数据
# 返回 no_grad_data: 不需要计算梯度，不需要进行反向传播的数据
def get_data(sup_batch, unsup_batch, config):
  # 定义成字典
  grad_data = {}
  no_grad_data = {}
  # sup_batch [bs, seq_len] bs:batchsize
  # unsup_batch [bs, seq_len]
  # 监督数据的 最长 长度
  sup_max_len = sup_batch['input_ids'].size(1)
  # 无监督数据 的最长 长度
  unsup_max_len = unsup_batch['ba_input_ids'].size(1)
  
  # 当前数据 的最长 长度(谁短补谁)
  cur_max_len = max(sup_max_len, unsup_max_len)
  # sup_batch是个字典
  for item, sup_value in sup_batch.items():
    # 如果键是标签 直接放进来
    if item == 'labels':
      grad_data[item] = sup_value.to(config['device'])
      continue

    ba_unsup_value = unsup_batch[f'ba_{item}']
    ab_unsup_value = unsup_batch[f'ab_{item}']
    
    # 谁短补谁，ba_unsup_value短
    if sup_max_len == cur_max_len:
      padding_value = torch.zeros((ba_unsup_value.size(0), cur_max_len - unsup_max_len), dtype = ba_unsup_value.dtype)
      ba_unsup_value = torch.cat([ba_unsup_value, padding_value], dim = -1)
    
    else:
      padding_value = torch.zeros((sup_value.size(0), cur_max_len - sup_max_len), dtype = sup_value.dtype)
      sup_value = torch.cat([sup_value, padding_value], dim = -1)
    
    # 把 sup_batch 和 ba 的 数据放在一起
    grad_value = torch.cat([sup_value, ba_unsup_value], dim = 0)
    grad_data[item] = grad_value.to(config['device'])
    # 对它不用进行操作直接放入字典  
    no_grad_data[item] = ab_unsup_value.to(config['device'])

  return grad_data, no_grad_data


In [31]:
# sharpen操作
import torch.nn as nn
logits = torch.randn(2,3)
print('logits:',logits)
t_softmax = torch.softmax(logits, dim=1)
# 一个[]中的数加起来为1
print('t_softmax:',t_softmax)
# 0.4 为config['uda_softmax_temp']
# 让大的更大小的更小，变得更硬标签
t_sharpen = torch.softmax(logits/0.4, dim=1)
print('t_sharpen:',t_sharpen)

logits: tensor([[ 0.1915,  0.3306,  0.2306],
        [ 0.8936, -0.2044, -0.9081]])
t_softmax: tensor([[0.3136, 0.3604, 0.3261],
        [0.6673, 0.2226, 0.1101]])
t_sharpen: tensor([[0.2842, 0.4024, 0.3134],
        [0.9300, 0.0597, 0.0103]])


#### no_grad_data 经过前向传播，产生ab的逻辑值，做ba逻辑值的标签

In [32]:
# 无监督数据 (ab) 只需要正向传播
# Confidence-based masking
def forward_no_grad(no_grad_data, config, model):
  with torch.no_grad():
    # 取逻辑值
    np_grad_logits = model(**no_grad_data)[0]
    # ----------- sharpen操作(config['uda_softmax_temp']) -------------#
    # 概率值
    no_grad_probs = torch.softmax(np_grad_logits / config['uda_softmax_temp'], dim = -1)
    # ----------- sharpen -------------#
    # 取出最大概率值
    # largest_probs [Batchsize] 举例：[0.879, 0.987, 0.234, 0.768, 0.333]
    largest_probs, _ = no_grad_probs.max(dim = -1)
    # config['uda_confidence_threshold']为0.7  是否大于0.7
    unsup_loss_mask = largest_probs.gt(config['uda_confidence_threshold']).float()
    # unsup_loss_mask tensor([True, True, False, True, False])

    
  return unsup_loss_mask, no_grad_probs

**与上面的图相差mask**
![在这里插入图片描述](https://img-blog.csdnimg.cn/b21349f49ba446b698e6f27823983fc5.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

**在forward_with_grad中reduction='none':**\
CE：交叉熵

![ce](https://img-blog.csdnimg.cn/52f18386dbea423f846611c558aa24c7.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

**CE交叉熵  E熵  KLD KL散度**

$$CE = -plogq$$   
$$E = -plogp$$
p-真实分布（已知） q-预测分布
$$KLDiv(p||q) = \sum_{i=1}^{N}p(x_{i})(logp(x_{i})- logq(x_{i}))$$
$$ = p(logp-logq) $$
$$ = -plogq - (- plogp)$$
$$ = CE-E$$

**KL散度就是拿交叉熵减去熵，熵又是确定的**


*torch接口中，class是类别的索引，是整数\
在无监督中,标签是ab_logits是非整数 因此用KL散度*

![cross entropy](https://img-blog.csdnimg.cn/4572c78d76624c49b01b96a1cba42279.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
![KL](https://img-blog.csdnimg.cn/189e4bc953904c199afbc7e6a11e5d9a.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [33]:
def forward_with_grad(unsup_loss_mask, unsup_probs, config, cur_bs, model, grad_data, total_steps, global_steps):
  # 得到\eta值， 随着训练的进行，阈值逐渐变大，最后是1，把所有监督数据都用上了
  tsa_threshold = get_tsa_threshold(total_steps, global_steps)
  # 逻辑值包括有监督的和无监督的
  logits = model(**grad_data)[0]
  # --------- 有监督损失 -------#
  # cur_bs 无监督 ba 的 batch_size
  # 前面一部分是有监督 train 的 sup_data, 后面是unsup_data
  sup_logits, unsup_logits = logits.split([logits.size(0) - cur_bs, cur_bs])
  # 得到 sup_labels
  sup_labels = grad_data['labels'][:logits.size(0) - cur_bs]
  # 计算交叉熵损失 （每个样本）
  per_example_loss = nn.CrossEntropyLoss(reduction = 'none')(sup_logits, sup_labels)
  # 拿出 正确标签 对应的概率
  correct_label_probs = torch.softmax(sup_logits, dim = -1).gather(dim = -1, index = sup_labels.view(-1, 1))
  # 监督数据 过于自信不要，留下小于等于 tsa_threshold 的计算损失
  sup_loss_mask = correct_label_probs.le(tsa_threshold).squeeze().float()
  # 应用mask掩盖有监督数据过度自信的样本损失
  per_example_loss *= sup_loss_mask

  # 有效监督样本的平均损失
  # (参考forward_no_grad)
  sup_loss = per_example_loss.sum() / max(sup_loss_mask.sum(), 1) # max(sup_loss_mask.sum(), 1) 有效个数
  # --------- 有监督损失 -------#

  # --------- 无监督损失 -------#
  # log_softmax：为了能输入到KL散度中
  unsup_log_probs = torch.log_softmax(unsup_logits, dim = -1)
  # input 希望是一个对数概率
  # Target 目标为概率值
  # ab_logits:unsup_probs
  per_example_kl_loss = nn.KLDivLoss(reduction = 'none')(unsup_log_probs, unsup_probs).sum(dim = -1)
  # 应用mask掩盖无监督数据中不自信的样本损失
  per_example_kl_loss *= unsup_loss_mask
  # 计算无监督样本的平均损失
  unsup_loss = per_example_kl_loss.sum() / max(unsup_loss_mask.sum(), 1)
  # --------- 无监督损失 -------#

  # 加权两种损失
  loss = sup_loss + unsup_loss
  # 多卡取平均
  if config['n_gpus'] > 1:
    loss = loss.mean()
    sup_loss = sup_loss.mean()
    unsup_loss = unsup_loss.mean()

  return loss, tsa_threshold, unsup_loss, sup_loss

In [34]:
from transformers import AdamW
from tqdm import trange
import os
def train(config, train_dataloader, dev_dataloader, unsup_dataloader=None):
  model = BertForAFQMC.from_pretrained(config['model_path'])

  optimizer = AdamW(model.parameters(), lr = config['learning_rate'], weight_decay = config['weight_decay'])

  model.to(config['device'])
  # unsup_dataloader包含 train, dev, test    
  # 使用 unsup_dataloader，因为unsup_dataloader比较大
  total_steps = len(unsup_dataloader) * config['num_epochs']
  epoch_iterator = trange(config['num_epochs'])
  global_steps = 0
  train_loss = 0.
  logging_loss = 0.
  best_acc = 0.
  UDA_model_path = ''

  if config['n_gpus'] > 1:
    model =nn.DataParallel(model)

  train_iterator = iter(train_dataloader)
  for i in epoch_iterator:
    unsup_iterator = tqdm(unsup_dataloader, desc = 'Training', total = len(unsup_dataloader))
    model.train()
    # ----------------------- new ----------------------#
    # 遍历无监督的batch
    for unsup_batch in unsup_iterator:
      cur_bs = unsup_batch['ab_input_ids'].size(0)
      try:
        # 不断拿监督数据的batch
        sup_batch = next(train_iterator)

      except StopIteration:
        # 监督数据用完了 再重新拿
        train_iterator = iter(train_dataloader)
        sup_batch = next(train_iterator)
      # 返回 grad_data：需要计算梯度，需要进行反向传播的数据
      # 返回 no_grad_data: 不需要计算梯度，不需要进行反向传播的数据
      grad_data, no_grad_data = get_data(sup_batch, unsup_batch, config)
      # 无监督数据 (ab) 只需要正向传播
      # mask, ab_logits
      unsup_loss_mask, unsup_probs =forward_no_grad(no_grad_data, config, model)
      # 得出loss
      loss, tsa_threshold, unsup_loss, sup_loss = forward_with_grad(unsup_loss_mask, unsup_probs, config, cur_bs, model, grad_data,
                                        total_steps, global_steps)
      model.zero_grad()
      loss.backward()

      nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
      optimizer.step()

      if config['ema_start']:
        ema.update()

      train_loss += loss.item()
      global_steps += 1

      unsup_iterator.set_postfix_str(f'running training loss: {loss.item(): .5f}')

      if global_steps % config['logging_step'] == 0:
        if global_steps >= config['ema_start_step'] and not config['ema_start']:
          print('\n>>> EMA starting .....')
          config['ema_start'] = True

          ema = EMA(model.module if hasattr(model, 'module') else model, decay = 0.99)

        print_train_loss = (train_loss - logging_loss) / config['logging_step']
        logging_loss = train_loss

        if config['ema_start']:
          ema.apply_shadow()
        val_loss, f1, acc = evaluation(config, model, dev_dataloader)

        print_log = f'\n>>> training loss: {print_train_loss:.6f}, valid loss: {val_loss:.6f},' 

        if acc > best_acc:
          model_save_path = os.path.join(config['output_path'],
                          f'checkpoint- {global_steps} - {acc:.6f}')
          model_to_save = model.module if hasattr(model, 'module') else model
          model_to_save.save_pretrained(model_save_path)
          best_acc = acc
          UDA_model_path = model_save_path
        print_log += f'valid f1: {f1:.6f}, valid acc:{acc:.6f}'

        print(print_log)
        model.train()

        if config['ema_start']:
          ema.restore()


  return model, UDA_model_path

In [35]:
model, UDA_model_path = train(config, train_dataloader, dev_dataloader, unsup_dataloader)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model were not used when initializing BertForAFQMC: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForAFQMC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForAFQMC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForAFQMC were not initialized from the model checkpoint at /content/drive/MyDrive/Cola


>>> training loss: 1.014281, valid loss: 0.659372,valid f1: 0.149620, valid acc:0.662882



Training:  34%|███▍      | 599/1772 [06:34<14:24,  1.36it/s, running training loss:  0.95734]


>>> EMA starting .....




Evaluation:   0%|          | 0/270 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/270 [00:00<01:46,  2.51it/s]

Evaluation:   1%|          | 2/270 [00:00<01:04,  4.16it/s]

Evaluation:   1%|          | 3/270 [00:00<00:48,  5.53it/s]

Evaluation:   1%|▏         | 4/270 [00:00<00:41,  6.35it/s]

Evaluation:   2%|▏         | 5/270 [00:00<00:37,  7.08it/s]

Evaluation:   2%|▏         | 6/270 [00:00<00:35,  7.51it/s]

Evaluation:   3%|▎         | 8/270 [00:01<00:29,  8.83it/s]

Evaluation:   3%|▎         | 9/270 [00:01<00:29,  8.94it/s]

Evaluation:   4%|▎         | 10/270 [00:01<00:28,  9.16it/s]

Evaluation:   4%|▍         | 12/270 [00:01<00:24, 10.37it/s]

Evaluation:   5%|▌         | 14/270 [00:01<00:26,  9.73it/s]

Evaluation:   6%|▌         | 15/270 [00:01<00:26,  9.75it/s]

Evaluation:   6%|▌         | 16/270 [00:02<00:27,  9.09it/s]

Evaluation:   6%|▋         | 17/270 [00:02<00:28,  8.87it/s]

Evaluation:   7%|▋         | 19/270 [00:02<00:26,  9.50it/s]

Evaluation:   7%|▋     

...


Training:  68%|██████▊   | 1199/1772 [13:55<05:41,  1.68it/s, running training loss:  1.21199]

Evaluation:   0%|          | 0/270 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/270 [00:00<01:36,  2.80it/s]

Evaluation:   1%|          | 2/270 [00:00<00:58,  4.56it/s]

Evaluation:   1%|          | 3/270 [00:00<00:44,  5.98it/s]

Evaluation:   1%|▏         | 4/270 [00:00<00:39,  6.73it/s]

Evaluation:   2%|▏         | 5/270 [00:00<00:35,  7.45it/s]

Evaluation:   2%|▏         | 6/270 [00:00<00:34,  7.76it/s]

Evaluation:   3%|▎         | 8/270 [00:01<00:29,  8.88it/s]

Evaluation:   3%|▎         | 9/270 [00:01<00:29,  8.70it/s]

Evaluation:   4%|▍         | 11/270 [00:01<00:26,  9.75it/s]

Evaluation:   5%|▍         | 13/270 [00:01<00:25, 10.03it/s]

Evaluation:   5%|▌         | 14/270 [00:01<00:26,  9.59it/s]

Evaluation:   6%|▌         | 16/270 [00:01<00:26,  9.57it/s]

Evaluation:   6%|▋         | 17/270 [00:02<00:27,  9.17it/s]

Evaluation:   7%|▋         | 19/270 [00:02<00:26,  9


>>> training loss: 0.989552, valid loss: 0.629359,valid f1: 0.000000, valid acc:0.689991



Training:  85%|████████▍ | 1499/1772 [17:34<02:56,  1.55it/s, running training loss:  0.94809]

Evaluation:   0%|          | 0/270 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/270 [00:00<01:39,  2.70it/s]

Evaluation:   1%|          | 2/270 [00:00<01:02,  4.30it/s]

Evaluation:   1%|          | 3/270 [00:00<00:47,  5.62it/s]

Evaluation:   1%|▏         | 4/270 [00:00<00:41,  6.47it/s]

Evaluation:   2%|▏         | 5/270 [00:00<00:36,  7.18it/s]

Evaluation:   2%|▏         | 6/270 [00:00<00:34,  7.58it/s]

Evaluation:   3%|▎         | 8/270 [00:01<00:29,  8.90it/s]

Evaluation:   3%|▎         | 9/270 [00:01<00:29,  8.76it/s]

Evaluation:   4%|▍         | 11/270 [00:01<00:26,  9.76it/s]

Evaluation:   5%|▍         | 13/270 [00:01<00:26,  9.84it/s]

Evaluation:   5%|▌         | 14/270 [00:01<00:27,  9.25it/s]

Evaluation:   6%|▌         | 16/270 [00:01<00:27,  9.36it/s]

Evaluation:   6%|▋         | 17/270 [00:02<00:28,  9.03it/s]

Evaluation:   7%|▋         | 19/270 [00:02<00:26,  9


>>> training loss: 0.978700, valid loss: 0.618604,valid f1: 0.000000, valid acc:0.689991



100%|██████████| 1/1 [20:59<00:00, 1259.24s/it]


In [36]:
from zipfile import ZipFile
def prediction(config, model, test_dataloader):
  test_iterator = tqdm(test_dataloader, desc = 'Prediction', total = len(test_dataloader))
  test_preds = []
  model.eval()

  with torch.no_grad():
    for batch in test_iterator:
      batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}

      logits = model(**batch_cuda)[0]

      probs = torch.softmax(logits, dim = -1)

      test_preds.append(probs[:, 1].detach().cpu())
    test_preds = torch.cat(test_preds)
    test_preds = torch.stack(test_preds.split(2), dim = 0).mean(dim = 1).numpy()
    submission_path = os.path.join(config['output_path'], 'submission_part2.tsv')
    test_df = pd.DataFrame(data = {'prediction': test_preds})
    test_df.to_csv(submission_path, index = False, header = False, encoding = 'utf8', sep = '\t')
    with ZipFile(os.path.join(config['output_path'], 'submission_part2.zip'), 'w') as myzip:
      myzip.write(submission_path, 'submission_part2.tsv')
      

In [38]:
prediction(config, model, test_dataloader)

Prediction: 100%|██████████| 483/483 [00:43<00:00, 11.00it/s]
